In [1]:
import hoomd
from hoomd import md

HOOMD-blue v2.1.1 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 10/24/16
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----


# Advancing simulations

Use `run` to advance a simulation forward a given number of time steps. For this tutorial, we use the same setup from the Quick Start.

In [2]:
hoomd.context.initialize('--mode=cpu')
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5)
nl = md.nlist.cell()
lj = md.pair.lj(r_cut=3.0, nlist=nl)
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
all = hoomd.group.all();
md.integrate.mode_standard(dt=0.001)
md.integrate.brownian(group=all, kT=0.1, seed=987)

HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values


After initialization with `create_lattice`, the simulation starts at time step 0. Query the current step counter with [get_step](http://hoomd-blue.readthedocs.io/en/stable/package-hoomd.html#hoomd.get_step).

In [3]:
hoomd.get_step()

0

Each HOOMD script can have one **integrator** active at a time. This example uses the `mode_standard` integrator from the `md` package. On each time step, `mode_standard` advances the simulation forward `dt` time units, integrating the equations of motion specified in the provided integration methods (brownian dynamics in this example). Use `run` to advance the simulation a given number of steps.

In [4]:
hoomd.run(1000)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:39 | Step 1000 / 1000 | TPS 1848.57 | ETA 00:00:00
Average TPS: 1846.76
---------
-- Neighborlist stats:
54 normal updates / 11 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 22 / n_neigh_avg: 9.392
shortest rebuild period: 12
-- Cell list stats:
Dimension: 2, 2, 2
n_min    : 11 / n_max: 19 / n_avg: 15.625
** run complete **


Now we are at step 1000.

In [5]:
hoomd.get_step()

1000

## Run options

The output from run is useful in batch jobs. However, sometimes you may want to supress it with the `quiet` option.

In [6]:
hoomd.run(1000, quiet=True)
hoomd.get_step()

2000

You may want to limit a run to a given length of time (e.g. for short benchmark runs). Use `limit_hours` to stop a run after so many hours, even if it has not reached the requested number of time steps.

In [7]:
hoomd.run(1e6, limit_hours=5/3600, quiet=True)

notice(2): Ending run at time step 74451 as 0.00138889 hours have passed


The profile option prints information on how much time was spent in each portion of the calculation. This is helpful when trying to identify why a simulation is running slowly.

In [8]:
hoomd.run(10000, profile=True)

** starting run **
Time 00:01:42 | Step 84451 / 84451 | TPS 13640.2 | ETA 00:00:00
Average TPS: 13635.8
Simulation:  0.7334s | 100.000% 
        Integrate:  0.2186s | 29.811% 
                BD step 1:  0.2008s | 27.384% 
                Net force:  0.0121s |  1.657% 
                Sum accel:  0.0000s |  0.001% 
                Self:       0.0056s |  0.770% 
        Neighbor:   0.0735s | 10.019% 
                Cell:        0.0017s |  0.237% 
                        compute:  0.0013s |  0.181% 
                        init:     0.0000s |  0.004% 
                Dist check:  0.0082s |  1.122% 
                compute:     0.0611s |  8.332% 
                head-list:   0.0000s |  0.006% 
                Self:        0.0024s |  0.322% 
        Pair lj:    0.4329s | 59.023% 
        SFCPack:    0.0023s |  0.316% 
        Self:       0.0061s |  0.831% 
---------
-- Neighborlist stats:
246 normal updates / 101 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 51 / n_ne

## Callbacks

You can specify an arbitrary python callback to run. This function will be called at the given period. If the function returns a negative value, the run is aborted.

In [9]:
def my_callback(step):
    print("This is step", step)
    if step >= 70000:
        return -1
    else:
        return 0

hoomd.run(1e6, callback=my_callback, callback_period=1000, quiet=True)

This is step 85000
notice(2): End of run requested by python callback at step 85000 / 1084451


In [11]:
def my_callback(step):
    print("This is step", step)
    if step >= 95000:
        return -1
    else:
        return 0

hoomd.run(1e6, callback=my_callback, callback_period=1000, profile=True)

** starting run **
This is step 85000
This is step 86000
This is step 87000
This is step 88000
This is step 89000
This is step 90000
This is step 91000
This is step 92000
This is step 93000
This is step 94000
This is step 95000
notice(2): End of run requested by python callback at step 95000 / 1085000
Time 00:03:46 | Step 95000 / 1085000 | TPS 13218.78813 | ETA 00:01:14
Average TPS: 13216.27240
Simulation:  0.7567s | 100.000% 
        Integrate:  0.2250s | 29.733% 
                BD step 1:  0.2041s | 26.968% 
                Net force:  0.0145s |  1.914% 
                Sum accel:  0.0000s |  0.000% 
                Self:       0.0064s |  0.851% 
        Neighbor:   0.0780s | 10.304% 
                Cell:        0.0021s |  0.279% 
                        compute:  0.0015s |  0.200% 
                        init:     0.0000s |  0.002% 
                Dist check:  0.0085s |  1.127% 
                compute:     0.0646s |  8.533% 
                head-list:   0.0001s |  0.008% 
     